In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Import Data
ords_prods_merge = pd.read_pickle("/Users/sarahboller/Desktop/CF 4 Instacart Basket Analysis/02 Data/Prepared Data/ords_prods_merge.pkl")

In [3]:
# Define Path for later
path = r'/Users/sarahboller/Desktop/CF 4 Instacart Basket Analysis'

In [4]:
# Step 2
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


In [ ]:
# Step 3 Difference between subset and full data
   #There are more departments listed in the full dataframe
    #The mean values in the subset are slightly higher/lower than the means found using the entire dataframe

In [5]:
# Step 4 Loyalty Flag 
 ## Transform
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

/var/folders/xq/j07ql7ls565cz8lmytmw14b40000gn/T/ipykernel_84893/53648537.py:2: FutureWarning: The provided callable <function amax at 0x11347cd60> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [11]:
## loc
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [12]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [13]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [14]:
# Check counts:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

In [15]:
ords_prods_merge[['user_id', 'loyalty_flag', 'order_number']].head(60)

,user_id,loyalty_flag,order_number
0,1,New customer,1
1,1,New customer,2
2,1,New customer,3
3,1,New customer,4
4,1,New customer,5
5,1,New customer,6
6,1,New customer,7
7,1,New customer,8
8,1,New customer,9
9,1,New customer,10


In [17]:
#Step 5 Aggregate  to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

In [22]:
# Step 6: Create Spending flag
ords_prods_merge['mean_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

/var/folders/xq/j07ql7ls565cz8lmytmw14b40000gn/T/ipykernel_84893/3342486745.py:2: FutureWarning: The provided callable <function mean at 0x11347da80> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['mean_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [26]:
ords_prods_merge.loc[ords_prods_merge['mean_price'] < 10, 'Spender_flag'] = 'Low Spender'


In [25]:
ords_prods_merge.loc[ords_prods_merge['mean_price'] >= 10, 'Spender_flag'] = 'High Spender'

In [27]:
ords_prods_merge['Spender_flag'].value_counts(dropna = False)

Spender_flag
Low Spender     31770614
High Spender      634245
Name: count, dtype: int64

In [33]:
ords_prods_merge[['user_id', 'mean_price', 'Spender_flag']].head(20)

,user_id,mean_price,Spender_flag
0,1,6.367797,Low Spender
1,1,6.367797,Low Spender
2,1,6.367797,Low Spender
3,1,6.367797,Low Spender
4,1,6.367797,Low Spender
5,1,6.367797,Low Spender
6,1,6.367797,Low Spender
7,1,6.367797,Low Spender
8,1,6.367797,Low Spender
9,1,6.367797,Low Spender


In [34]:
# Step 7 Create frequency flag
ords_prods_merge['median_freq'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

/var/folders/xq/j07ql7ls565cz8lmytmw14b40000gn/T/ipykernel_84893/2601665436.py:2: FutureWarning: The provided callable <function median at 0x1135bbd80> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['median_freq'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [36]:
ords_prods_merge.loc[ords_prods_merge['median_freq'] > 20, 'freq_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median_freq'] > 10) & (ords_prods_merge['median_freq'] <= 20), 'freq_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['median_freq'] <= 10, 'freq_flag'] = 'Frequent customer'

In [40]:
ords_prods_merge[['user_id','median_freq','freq_flag']].head(20)

,user_id,median_freq,freq_flag
0,1,20.5,Non-frequent customer
1,1,20.5,Non-frequent customer
2,1,20.5,Non-frequent customer
3,1,20.5,Non-frequent customer
4,1,20.5,Non-frequent customer
5,1,20.5,Non-frequent customer
6,1,20.5,Non-frequent customer
7,1,20.5,Non-frequent customer
8,1,20.5,Non-frequent customer
9,1,20.5,Non-frequent customer


In [41]:
ords_prods_merge['freq_flag'].value_counts(dropna = False)

freq_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: count, dtype: int64

In [43]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_combined.pkl'))